In [2]:
import pandas as pd
import random
import copy
import os
import importlib
import sys
from collections import Counter, defaultdict
from deap import base, creator, tools

# --- Konfigurasi Global ---
DATASET_CATEGORIES = {
    "Low": "JADWAL PERKULIAHAN SEMESTER PENDEK  T.A 23_24 - Jadwal.csv",
    "Medium": "JADWAL PERKULIAHAN SEMESTER GENAP 2023_2024 - Jadwal.csv"
}
DAYS_ORDER = ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']

print("✅ Pustaka dan Konfigurasi Awal Siap.")

# ==============================================================================
# BAGIAN 2: FUNGSI-FUNGSI INTI (PARSING, EVALUASI, PREFERENSI)
# ==============================================================================

def parse_schedule_dynamically(file_path):
    """Fungsi parsing CSV yang paling tangguh."""
    try:
        header_df = pd.read_csv(file_path, header=None, nrows=7, engine='python', dtype=str).fillna('')
        day_positions, header_row_index = {}, -1
        for idx, row in header_df.iterrows():
            if any(day in row.astype(str).values for day in DAYS_ORDER): header_row_index = idx; break
        if header_row_index == -1: return None, None, "❌ Error: Tidak dapat menemukan header hari."
        header_series = header_df.iloc[header_row_index]
        for col_idx, value in header_series.items():
            if isinstance(value, str):
                for day in DAYS_ORDER:
                    if day.lower() in value.lower(): day_positions[day] = col_idx; break
        jadwal_df = pd.read_csv(file_path, skiprows=header_row_index + 1, header=None, engine='python', dtype=str).fillna('')
        all_sessions, lecture_schedule, sesi_col, day_cols_template = [], [], 0, ['Kode MK', 'Mata Kuliah', 'T/P', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'I1', 'I2', 'Kelas', 'Ruang']
        for day in DAYS_ORDER:
            if day not in day_positions: continue
            start_col = day_positions[day]
            if start_col + len(day_cols_template) > len(jadwal_df.columns): continue
            day_df = jadwal_df.iloc[:, [sesi_col] + list(range(start_col, start_col + len(day_cols_template)))]
            day_df.columns = ['Sesi'] + day_cols_template
            day_df = day_df[day_df['Kode MK'].str.strip() != '']
            day_df = day_df[pd.to_numeric(day_df['Sesi'], errors='coerce').notna()]
            for idx, row in day_df.iterrows():
                teachers = [str(row[f'D{i}']).strip() for i in range(1, 8) if str(row[f'D{i}']).strip() not in ['', '-']] + \
                           [str(row[f'I{i}']).strip() for i in range(1, 3) if str(row[f'I{i}']).strip() not in ['', '-']]
                if not teachers or not str(row['Kelas']).strip() or not str(row['Ruang']).strip(): continue
                class_id = f"{str(row['Kode MK']).strip()}_{idx}_{str(row['Kelas']).strip()}"
                all_sessions.append({
                    'class_id': class_id,
                    'room': str(row['Ruang']).strip(),
                    'course_id': str(row['Kode MK']).strip(),
                    'teacher1': teachers[0] if teachers else None,
                    'type': str(row['T/P']).strip(),
                    'teacher2': teachers[1] if len(teachers) > 1 else None,
                    'group': str(row['Kelas']).strip(),
                    'day': day,
                    'session': str(row['Sesi']).strip(),
                    'course_name': str(row['Mata Kuliah']).strip()
                })
                lecture_schedule.append([
                    len(lecture_schedule) + 1,  # lecture_id
                    class_id,                   # class_id
                    str(row['Kode MK']).strip(), # course_id
                    teachers[0] if teachers else '0', # teacher_id1
                    teachers[1] if len(teachers) > 1 else '0' # teacher_id2
                ])
        print(f"✅ Berhasil mem-parsing {len(all_sessions)} sesi kuliah valid.")
        return all_sessions, lecture_schedule, None
    except Exception as e:
        return None, None, f"❌ Terjadi kesalahan kritis saat parsing: {e}"

def convert_to_random_schedule(schedule_list):
    """
    Mengonversi schedule_list ke format random_schedule yang BENAR
    untuk dikonsumsi oleh fungsi preferensi buatan AI.
    """
    random_schedule = []
    for session in schedule_list:
        room_id = session.get('room', '0')
        day = session.get('day')
        session_str = session.get('session', '-1')
        group_name = session.get('group', '0')
        course_id = session.get('course_id', '0')
        teacher1 = session.get('teacher1', '0')
        teacher2 = session.get('teacher2', '0')
        day_idx = DAYS_ORDER.index(day) if day in DAYS_ORDER else -1
        session_idx = int(session_str) - 1 if session_str.isdigit() else -1
        random_schedule.append([
            room_id, day_idx, session_idx, group_name, course_id, teacher1, teacher2
        ])
    return random_schedule

def count_hard_conflicts(schedule_list):
    """Mengevaluasi konflik dengan aturan khusus yang telah ditetapkan."""
    room_conflicts, teacher_conflicts, group_conflicts = 0, 0, 0
    slots = defaultdict(list)
    for session in schedule_list: slots[(session['day'], session['session'])].append(session)
    for slot_key, sessions_in_slot in slots.items():
        if len(sessions_in_slot) <= 1: continue
        teacher_events, room_events, group_usage = Counter(), Counter(), Counter()
        events = defaultdict(list)
        for s in sessions_in_slot:
            event_key = (s.get('teacher1'), s.get('room'), s.get('course_id')); events[event_key].append(s); group_usage[s.get('group')] += 1
        for (teacher, room, course), event_sessions in events.items():
            if teacher: teacher_events[teacher] += 1
            if room: room_events[room] += 1
        for teacher, count in teacher_events.items():
            if count > 4: teacher_conflicts += (count - 4)
        for room, count in room_events.items():
            if room == 'AUD':
                if count > 3: room_conflicts += (count - 3)
            elif count > 1: room_conflicts += (count - 1)
        for group, count in group_usage.items():
            if count > 1: group_conflicts += (count - 1)
    return (room_conflicts, teacher_conflicts, group_conflicts)

def generate_preference_function(user_prompt):
    """Mengimpor fungsi preferensi dari file buatan AI."""
    print(f"\nMenggunakan preferensi: '{user_prompt}'")
    print(f"   - Mengimpor fungsi preferensi dari lecturer_preference_function, student_preference_function, institutional_preference_function...")
    try:
        import lecturer_preference_function
        import student_preference_function
        import institutional_preference_function
        importlib.reload(lecturer_preference_function)
        importlib.reload(student_preference_function)
        importlib.reload(institutional_preference_function)
        from lecturer_preference_function import count_preference_conflict as lecturer_conflict
        from student_preference_function import count_preference_conflict as student_conflict
        from institutional_preference_function import count_preference_conflict as institutional_conflict

        def wrapped_preference_function(schedule_list, lecture_schedule):
            random_schedule = convert_to_random_schedule(schedule_list)
            lecturer_score = lecturer_conflict(random_schedule, lecture_schedule)
            student_score = student_conflict(random_schedule, lecture_schedule)
            institutional_score = institutional_conflict(random_schedule, lecture_schedule)
            return lecturer_score, student_score, institutional_score

        print("   - ✅ Fungsi preferensi berhasil diimpor.")
        return wrapped_preference_function, user_prompt
    except Exception as e:
        print(f"   - ❌ Gagal mengimpor fungsi preferensi: {e}.")
        return (lambda s, l: (-1, -1, -1)), user_prompt

# ==============================================================================
# BAGIAN 3: FUNGSI OPTIMISASI DAN ANALISIS
# ==============================================================================

def mutTargeted(individual, preference_function, lecture_schedule, indpb, all_rooms, all_sessions_options):
    """Operator mutasi cerdas yang menargetkan sesi yang konflik."""
    conflicting_indices = [i for i, session in enumerate(individual) if sum(preference_function([session], lecture_schedule)) > 0]

    if not conflicting_indices or random.random() > indpb:
        return mutMoveToEmptySlot(individual, 0.05, all_rooms, all_sessions_options)

    idx_to_move = random.choice(conflicting_indices)
    original_session = individual[idx_to_move]

    occupied_slots = set((s['day'], s['session'], s['room']) for s in individual)

    for _ in range(200):
        new_day, new_session, new_room = random.choice(DAYS_ORDER), random.choice(all_sessions_options), random.choice(all_rooms)
        if (new_day, new_session, new_room) not in occupied_slots:
            individual[idx_to_move]['day'] = new_day
            individual[idx_to_move]['session'] = new_session
            individual[idx_to_move]['room'] = new_room
            return individual,

    return individual,

def mutMoveToEmptySlot(individual, indpb, all_rooms, all_sessions_options):
    """Mutasi acak sebagai cadangan."""
    occupied_slots = set((s['day'], s['session'], s['room']) for s in individual)
    for i in range(len(individual)):
        if random.random() < indpb:
            for _ in range(100):
                new_day, new_session, new_room = random.choice(DAYS_ORDER), random.choice(all_sessions_options), random.choice(all_rooms)
                if (new_day, new_session, new_room) not in occupied_slots:
                    individual[i]['day'], individual[i]['session'], individual[i]['room'] = new_day, new_session, new_room
                    break
    return individual,

def setup_deap_toolbox_nsga3(initial_schedule, lecture_schedule, preference_function):
    """Mempersiapkan toolbox DEAP dengan fungsi evaluasi dan mutasi yang diperbaiki."""
    if hasattr(creator, "FitnessMin"): del creator.FitnessMin
    if hasattr(creator, "Individual"): del creator.Individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0, -1.0, -1.0))  # Hard + 3 soft conflicts
    creator.create("Individual", list, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initIterate, creator.Individual, lambda: copy.deepcopy(initial_schedule))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    def evaluate(individual):
        hard_conflicts = sum(count_hard_conflicts(individual))
        lecturer_score, student_score, institutional_score = preference_function(individual, lecture_schedule)
        PENALTY = 10000
        penalized_hard_fitness = hard_conflicts * PENALTY
        return (penalized_hard_fitness, lecturer_score, student_score, institutional_score)
    toolbox.register("evaluate", evaluate)

    toolbox.register("mate", tools.cxTwoPoint)

    all_rooms = list(set(s['room'] for s in initial_schedule))
    all_sessions_options = list(set(s['session'] for s in initial_schedule))

    toolbox.register("mutate", mutTargeted,
                     preference_function=preference_function,
                     lecture_schedule=lecture_schedule,
                     indpb=0.5,
                     all_rooms=all_rooms,
                     all_sessions_options=all_sessions_options)
    
    ref_points = tools.uniform_reference_points(nobj=4, p=12)  # Updated for 4 objectives
    toolbox.register("select", tools.selNSGA3, ref_points=ref_points)
    return toolbox

def run_optimization_nsga3(initial_schedule, lecture_schedule, preference_function, pop_size, generations, cxpb, mutpb):
    """Menjalankan algoritma genetika NSGA-III dengan strategi Elitism yang diperbaiki."""
    toolbox = setup_deap_toolbox_nsga3(initial_schedule, lecture_schedule, preference_function)
    pop = toolbox.population(n=pop_size)
    print("  -> Mengevaluasi populasi awal untuk optimisasi...")
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses): ind.fitness.values = fit

    best_known_individual = copy.deepcopy(tools.selBest(pop, 1)[0])
    if sum(count_hard_conflicts(best_known_individual)) > 0:
        print("⚠️ Peringatan: Jadwal awal yang menjadi dasar elitisme sudah memiliki konflik hard.")

    print(f"\n🚀 Memulai Optimisasi NSGA-III... (Pop: {pop_size}, Gen: {generations}, CX: {cxpb}, Mut: {mutpb})")
    for gen in range(1, generations + 1):
        offspring = toolbox.select(pop, len(pop)); offspring = list(map(toolbox.clone, offspring))
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb: toolbox.mate(child1, child2); del child1.fitness.values; del child2.fitness.values
        for mutant in offspring:
            if random.random() < mutpb: toolbox.mutate(mutant); del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]; fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses): ind.fitness.values = fit

        for ind in pop + offspring:
            if sum(ind.fitness.values) < sum(best_known_individual.fitness.values):
                best_known_individual = copy.deepcopy(ind)

        pop[:] = toolbox.select(pop + offspring, pop_size)
        pop[0] = best_known_individual

        best_hard, best_lecturer, best_student, best_institutional = best_known_individual.fitness.values
        best_hard_real = best_hard / 10000

        if gen % 100 == 0:
            print(f"  -> Gen {gen}/{generations} | Solusi Terbaik Sejauh Ini: Hard={int(best_hard_real)}, "
                  f"Lecturer={best_lecturer}, Student={best_student}, Institutional={best_institutional}")
            if best_hard_real == 0 and best_lecturer == 0 and best_student == 0 and best_institutional == 0:
                print("🎉 Solusi optimal (Hard=0, Lecturer=0, Student=0, Institutional=0) ditemukan!"); break

    print("✅ Optimisasi Selesai.")
    return best_known_individual

def analyze_and_compare_schedules(initial_schedule, final_schedule, lecture_schedule, pref_function, user_prompt, category):
    """Menganalisis dan membandingkan jadwal dengan ringkasan yang lebih cerdas."""
    print("\n" + "="*80)
    print(f"ANALISIS KOMPARATIF: JADWAL AWAL vs JADWAL HASIL OPTIMISASI (Kategori: {category})")
    print("="*80)
    initial_hard = sum(count_hard_conflicts(initial_schedule))
    initial_lecturer, initial_student, initial_institutional = pref_function(initial_schedule, lecture_schedule)
    final_hard = sum(count_hard_conflicts(final_schedule))
    final_lecturer, final_student, final_institutional = pref_function(final_schedule, lecture_schedule)
    print(f"\nPreferensi yang dioptimalkan: '{user_prompt}' (Kategori: {category})")
    print(f"\n--- Perbandingan Skor Konflik ---")
    print(f"{'Jenis Konflik':<30} | {'Jadwal Awal':<15} | {'Jadwal Optimasi':<15}")
    print("-" * 70)
    print(f"{'Konflik Hard (Total)':<30} | {initial_hard:<15} | {final_hard:<15}")
    print(f"{'Konflik Dosen':<30} | {initial_lecturer:<15} | {final_lecturer:<15}")
    print(f"{'Konflik Mahasiswa':<30} | {initial_student:<15} | {final_student:<15}")
    print(f"{'Konflik Institusi':<30} | {initial_institutional:<15} | {final_institutional:<15}")
    print("-" * 70)
    print("\n--- Ringkasan ---")
    if final_hard > initial_hard:
        print(f"⚠️ PERINGATAN! Optimisasi menambah {final_hard - initial_hard} konflik hard baru! Hasil ini TIDAK disarankan.")
    elif final_hard < initial_hard:
        print(f"✅ LUAR BIASA! Optimisasi berhasil menghilangkan {initial_hard - final_hard} konflik hard.")
    total_initial_soft = initial_lecturer + initial_student + initial_institutional
    total_final_soft = final_lecturer + final_student + final_institutional
    if final_hard == initial_hard and total_final_soft < total_initial_soft:
        print(f"✅ BAIK! Optimisasi berhasil mengurangi {total_initial_soft - total_final_soft} pelanggaran preferensi tanpa menambah konflik hard.")
    else:
        print("ℹ️ Tidak ada peningkatan signifikan. Jadwal mungkin sudah optimal atau perlu lebih banyak generasi/parameter berbeda.")

def export_final_schedule(final_schedule, category, output_file="optimized_schedule_{category}.csv"):
    """Prints the final schedule with group information and saves it to a CSV file."""
    session_times = {
        '1': '08:00-08:50',
        '2': '09:00-09:50',
        '3': '10:00-10:50',
        '4': '11:00-11:50',
        '5': '13:00-13:50',
        '6': '14:00-14:50',
        '7': '15:00-15:50',
        '8': '16:00-16:50'
    }

    sessions = sorted(set(s['session'] for s in final_schedule if s['session'] in session_times), key=lambda x: int(x))
    schedule_dict = defaultdict(list)
    for session in final_schedule:
        if session['session'] in session_times:
            schedule_dict[(session['day'], session['session'])].append(session)

    print("\n" + "="*80)
    print(f"FINAL OPTIMIZED SCHEDULE (DAFTAR PER HARI) - Kategori: {category}")
    print("="*80)

    for day in DAYS_ORDER:
        print(f"\n{day}:")
        for session in sessions:
            time_slot = session_times[session]
            key = (day, session)
            if key in schedule_dict:
                for s in schedule_dict[key]:
                    teachers = f"{s['teacher1']}" if s['teacher1'] else ""
                    if s['teacher2'] and s['teacher2'] != '0':
                        teachers += f", {s['teacher2']}"
                    info = f"Course {s['course_name']} (Teacher {teachers}, Room {s['room']}, Group {s['group']})"
                    print(f"  - {time_slot} → {info}")
            else:
                print(f"  - {time_slot} → No class scheduled")
    print("="*80)

    df = pd.DataFrame(final_schedule)
    df['time_slot'] = df['session'].map(session_times)
    df['teachers'] = df.apply(
        lambda row: f"{row['teacher1']}" + (f", {row['teacher2']}" if row.get('teacher2') and row['teacher2'] != '0' else ""),
        axis=1
    )
    df['group'] = df['group'].fillna('Unknown')
    output_df = df[['day', 'time_slot', 'course_name', 'teachers', 'room', 'group', 'course_id', 'class_id', 'type']]
    output_file = output_file.format(category=category)
    output_dir = os.path.dirname(output_file)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
    try:
        output_df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"\n✅ Schedule successfully saved to '{output_file}'")
    except Exception as e:
        print(f"\n❌ Error saving CSV: {e}")

def main_workflow(category="Low", user_prompt="Default preference"):
    print(f"--- LANGKAH 1: MEMBACA & MEM-PARSING JADWAL AWAL (Kategori: {category}) ---")
    if category not in DATASET_CATEGORIES:
        print(f"❌ Kategori '{category}' tidak valid. Pilih dari {list(DATASET_CATEGORIES.keys())}.")
        return
    file_path = DATASET_CATEGORIES[category]
    schedule_list, lecture_schedule, error = parse_schedule_dynamically(file_path)
    if error:
        print(error)
        return

    (r_conf, t_conf, g_conf) = count_hard_conflicts(schedule_list)
    total_hard_conflicts = r_conf + t_conf + g_conf
    print(f"\nVerifikasi Konflik Hard Awal: Total = {total_hard_conflicts} (Ruang: {r_conf}, Dosen: {t_conf}, Grup: {g_conf})")

    print("\n" + "="*80)
    print(f"--- LANGKAH 2: EVALUASI PREFERENSI (Kategori: {category}) ---")
    print("="*80)

    preference_function, pref_prompt = generate_preference_function(user_prompt)
    lecturer_score, student_score, institutional_score = preference_function(schedule_list, lecture_schedule)
    if lecturer_score >= 0 and student_score >= 0 and institutional_score >= 0:
        print(f"   -> Hasil Verifikasi: Ditemukan pelanggaran untuk preferensi '{pref_prompt}':")
        print(f"      - Konflik Dosen: {lecturer_score}")
        print(f"      - Konflik Mahasiswa: {student_score}")
        print(f"      - Konflik Institusi: {institutional_score}")
    else:
        print(f"   -> ❌ Gagal mengevaluasi preferensi '{pref_prompt}'.")
        return

    print("\n" + "="*80)
    print(f"--- LANGKAH 3: OPTIMISASI JADWAL (Kategori: {category}) ---")
    print("="*80)
    print(f"\n### MEMULAI OPTIMISASI UNTUK PREFERENSI: '{pref_prompt}' (Kategori: {category}) ###")
    final_schedule = run_optimization_nsga3(
        initial_schedule=schedule_list,
        lecture_schedule=lecture_schedule,
        preference_function=preference_function,
        pop_size=200,
        generations=1000,
        cxpb=0.9,
        mutpb=0.3
    )
    if final_schedule:
        analyze_and_compare_schedules(schedule_list, final_schedule, lecture_schedule, preference_function, pref_prompt, category)
        output_file = f"optimized_schedule_{category}.csv"
        export_final_schedule(final_schedule, category, output_file)

    print("\n" + "="*80)
    print(f"PROSES OPTIMISASI DAN EKSPOR JADWAL TELAH SELESAI (Kategori: {category}).")
    print("="*80)

if __name__ == '__main__':
    for category in DATASET_CATEGORIES.keys():
        main_workflow(category=category, user_prompt="Minimize teacher conflicts and prioritize morning sessions")

✅ Pustaka dan Konfigurasi Awal Siap.
--- LANGKAH 1: MEMBACA & MEM-PARSING JADWAL AWAL (Kategori: Low) ---
✅ Berhasil mem-parsing 144 sesi kuliah valid.

Verifikasi Konflik Hard Awal: Total = 124 (Ruang: 26, Dosen: 0, Grup: 98)

--- LANGKAH 2: EVALUASI PREFERENSI (Kategori: Low) ---

Menggunakan preferensi: 'Minimize teacher conflicts and prioritize morning sessions'
   - Mengimpor fungsi preferensi dari lecturer_preference_function, student_preference_function, institutional_preference_function...
   - ✅ Fungsi preferensi berhasil diimpor.
   -> Hasil Verifikasi: Ditemukan pelanggaran untuk preferensi 'Minimize teacher conflicts and prioritize morning sessions':
      - Konflik Dosen: 51
      - Konflik Mahasiswa: 0
      - Konflik Institusi: 0

--- LANGKAH 3: OPTIMISASI JADWAL (Kategori: Low) ---

### MEMULAI OPTIMISASI UNTUK PREFERENSI: 'Minimize teacher conflicts and prioritize morning sessions' (Kategori: Low) ###
  -> Mengevaluasi populasi awal untuk optimisasi...
⚠️ Peringatan: J

✅ Berhasil mem-parsing 810 sesi kuliah valid.

Verifikasi Konflik Hard Awal: Total = 803 (Ruang: 59, Dosen: 0, Grup: 744)

--- LANGKAH 2: EVALUASI PREFERENSI (Kategori: Medium) ---

Menggunakan preferensi: 'Minimize teacher conflicts and prioritize morning sessions'
   - Mengimpor fungsi preferensi dari lecturer_preference_function, student_preference_function, institutional_preference_function...
   - ✅ Fungsi preferensi berhasil diimpor.
   -> Hasil Verifikasi: Ditemukan pelanggaran untuk preferensi 'Minimize teacher conflicts and prioritize morning sessions':
      - Konflik Dosen: 222
      - Konflik Mahasiswa: 2
      - Konflik Institusi: 0

--- LANGKAH 3: OPTIMISASI JADWAL (Kategori: Medium) ---

### MEMULAI OPTIMISASI UNTUK PREFERENSI: 'Minimize teacher conflicts and prioritize morning sessions' (Kategori: Medium) ###
  -> Mengevaluasi populasi awal untuk optimisasi...
⚠️ Peringatan: Jadwal awal yang menjadi dasar elitisme sudah memiliki konflik hard.

🚀 Memulai Optimisasi NSGA-